## Experiments: Sensitive Analysis Hyperparameter RSAST:


In [1]:
from sast.sast import *
from cd_diagram.cd_function import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


In [56]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [58]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS'], dtype=object)

In [59]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)

In [60]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['SmoothSubspace', 'Chinatown', 'ItalyPowerDemand', 'Fungi',
       'ShapeletSim', 'BirdChicken'], dtype=object)

In [61]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Accuracy: Hypertunning RSAST 

It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

#### Accuracy: Subsequence Lenght Method 

In [87]:
# create a pivot table with the mean of score by hyperparameter
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.794000,0.826000,0.726000,0.744000,0.714000,0.830000,BirdChicken,0.900000
1,Chinatown,0.947697,0.961866,0.950087,0.958542,0.938950,0.960000,Chinatown,0.982507
2,Fungi,0.109570,0.999677,0.111290,0.916452,0.868817,0.998710,NaN,NaN
3,ItalyPowerDemand,0.924879,0.932692,0.927386,0.924937,0.925948,0.934266,ItalyPowerDemand,0.968902
4,ShapeletSim,0.961333,0.892444,0.893556,0.724667,0.661111,0.887556,ShapeletSim,1.000000
5,SmoothSubspace,0.820000,0.861244,0.820444,0.849333,0.823733,0.856267,SmoothSubspace,0.980000


In [29]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.698535,0.750164,0.754792,0.758864,0.755655
ACF&PACF,0.886091,0.923249,0.922479,0.926348,0.928491
Max ACF,0.683786,0.728502,0.753107,0.746930,0.750592
Max PACF,0.804481,0.884978,0.891733,0.894557,0.895401
None,0.796974,0.852589,0.866560,0.851661,0.865735
PACF,0.890524,0.912257,0.924823,0.925726,0.924991


In [88]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.663114,0.742436,0.765195,0.773630,0.773635
ACF&PACF,0.864525,0.917593,0.932923,0.933574,0.938044
Max ACF,0.648122,0.729767,0.762625,0.757340,0.765063
Max PACF,0.796106,0.874688,0.897715,0.899316,0.903325
None,0.763707,0.843493,0.862592,0.870503,0.893224
PACF,0.861163,0.921676,0.929211,0.932496,0.933775


#### Time: Subsequence Lenght Method 

In [32]:
# create a pivot table with the variance of accuracy by hyperparameter
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Fungi,ItalyPowerDemand,ShapeletSim,SmoothSubspace
len_method,,,,,,
ACF,84.041194,0.114219,4.672687,0.572094,7.662884,1.700110
ACF&PACF,306.721389,0.310967,19.247313,1.496879,220.645132,2.850623
Max ACF,2.835691,0.116253,0.888735,0.575331,2.680689,1.705966
Max PACF,117.303855,0.132535,2.241267,0.668883,122.725377,1.978221
None,1.925235,0.113482,0.532585,0.557837,1.915527,1.679851
PACF,214.689082,0.308953,14.731980,1.479227,212.739967,2.778925


In [34]:
# create a pivot table with the variance of accuracy by hyperparameter
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,1.764964,11.172157,11.778861,12.085262,12.072578
ACF&PACF,9.397083,60.670729,62.378948,67.605693,62.687870
Max ACF,0.282487,1.012097,1.428661,1.768142,1.779028
Max PACF,2.774062,24.578220,34.041680,26.958558,26.215327
None,0.201295,0.731294,1.168528,1.543292,1.486556
PACF,7.362183,49.747850,50.428586,52.666664,52.888419


In [35]:
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.341073,2.563792,7.763710,12.790557,25.414691
ACF&PACF,19.594841,27.815093,46.279881,65.410514,103.639993
Max ACF,0.112435,0.391544,0.996106,1.620256,3.150075
Max PACF,18.618618,19.466671,24.294951,26.534766,25.652842
None,0.082551,0.314267,0.813520,1.328436,2.592190
PACF,19.574235,25.716918,42.387119,52.959863,72.455566


#### Variance: Subsequence Lenght Method 

In [36]:
# create a pivot table with the mean of computation time in seconds by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.085467,0.099806,0.102758,0.104603,0.102218
ACF&PACF,0.015708,0.004403,0.002576,0.002837,0.002385
Max ACF,0.083217,0.097396,0.102697,0.102825,0.100869
Max PACF,0.024003,0.006959,0.006310,0.006369,0.006348
None,0.024217,0.016566,0.014860,0.015429,0.008801
PACF,0.014250,0.003805,0.003309,0.002644,0.002702


In [37]:
# create a pivot table with the mean of computation time in seconds by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.105236,0.102475,0.098067,0.096820,0.098541
ACF&PACF,0.013327,0.004558,0.004496,0.004452,0.003602
Max ACF,0.100810,0.106684,0.096089,0.097858,0.092044
Max PACF,0.019083,0.009962,0.008215,0.008172,0.006576
None,0.028105,0.017672,0.011140,0.016536,0.013136
PACF,0.009323,0.007259,0.004148,0.004095,0.004823


In [38]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,1650,1650,1650
unique,6,6,150
top,SmoothSubspace,ACF&PACF,ACF&PACF: n_random_points=1 nb_inst_per_class=1
freq,450,275,11


In [11]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,5.052198
Chinatown,0.015228
Fungi,0.587702
ItalyPowerDemand,0.074309
ShapeletSim,3.947011
SmoothSubspace,0.264452


In [12]:
df_result['time'].sum()/(60*60)

9.940899730655882

In [ ]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

In [ ]:
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='cd-diagram_MaxPACF.png')
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)


In [15]:
average_ranks

,0
ACF: n_random_points=1 nb_inst_per_class=1,142.833333
Max ACF: n_random_points=1 nb_inst_per_class=1,141.333333
Max ACF: n_random_points=10 nb_inst_per_class=1,136.916667
Max PACF: n_random_points=1 nb_inst_per_class=1,136.416667
None: n_random_points=1 nb_inst_per_class=1,131.750000
...,...
ACF&PACF: n_random_points=30 nb_inst_per_class=100,38.583333
PACF: n_random_points=50 nb_inst_per_class=100,38.333333
ACF&PACF: n_random_points=50 nb_inst_per_class=50,36.000000
PACF: n_random_points=100 nb_inst_per_class=100,35.750000


In [16]:
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=50 nb_inst_per_class=100'], dtype='object')

#### Focus on most accurate lenght method

In [17]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.761371,0.918189,0.917092,0.907658,0.926143
10,0.885603,0.923630,0.934539,0.935966,0.936509
30,0.877236,0.918289,0.936508,0.937934,0.942430
50,0.888394,0.914126,0.938629,0.945548,0.945043
100,0.910022,0.913731,0.937846,0.940763,0.940094


In [66]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.036118,0.004190,0.003213,0.005134,0.003079
10,0.011920,0.004333,0.002346,0.002298,0.002454
30,0.009942,0.004904,0.002712,0.002542,0.002187
50,0.009556,0.005983,0.002443,0.001962,0.002117
100,0.005603,0.006225,0.002531,0.002480,0.002407


##### Overall time complexity

In [18]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,2.180491,13.009470,13.568603,12.190020,12.443177
10,2.959124,18.267934,18.812651,18.828155,20.435658
30,6.518380,32.743741,36.812957,38.715769,40.672271
50,9.637075,50.281322,54.536493,49.662344,49.550662
100,17.755197,83.389445,84.298846,119.386456,86.802700


##### Calculate weights time complexity

In [11]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.028192,0.028556,0.027600,0.028790,0.030082
10,0.029349,0.025693,0.025898,0.029354,0.024608
30,0.022716,0.031543,0.029285,0.034425,0.029611
50,0.029784,0.026948,0.028282,0.030720,0.031881
100,0.029638,0.027652,0.028193,0.030118,0.025571


##### Finding subsequences time complexity

In [12]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.570715,0.450227,0.526023,0.500756,0.546674
10,0.467190,0.462596,0.496268,0.511135,0.533021
30,0.549582,0.544082,0.676318,0.758593,0.745421
50,0.694216,0.721292,0.873402,0.976438,1.038515
100,0.948055,1.015550,1.283563,1.370078,1.323146


##### Transform Dataset time complexity

In [13]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.428162,0.123954,0.143920,0.156997,0.153823
10,1.133522,1.208636,1.386036,1.510926,1.508498
30,3.357759,3.566634,4.144730,4.521114,4.506077
50,5.545827,5.986823,6.918355,7.589233,7.518632
100,11.129992,11.864132,13.768211,15.064077,14.992503


##### Classifier time complexity

In [14]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.003726,0.007340,0.012438,0.009509,0.012228
10,0.004278,0.011168,0.011942,0.010837,0.009700
30,0.011958,0.017603,0.013131,0.015574,0.013850
50,0.011572,0.013118,0.015163,0.020041,0.015440
100,0.014387,0.015632,0.024309,0.031184,0.024854
